This is the tutorial to show how to evaluate the RAG's accuracy metrics by using haystack's key value store. We have a customized retriver and special domain knowledge, create the RAG under haystack pipeline, and  evaluate the RAG by using a packed evaluation class. The results are saved in standarlized csv files, for the convinience of both human and machine.
- Dataset: one access defence news from janes
- Metrics: bleu and rouge, for both the ending results and intermediate prompts (by using haystack's K-V in evaluator classs)

Functions to load janes data

In [1]:
from janes.janesRead import loadCategories
pages=100
categories = ['weapons-news-list','land-news-list','air-news-list','sea-news-list']
def genQueryAndCorpus(t,b):
    queries=[]
    corpus=[]
    for i in range(len(t)):
        title=t[i]
        brief=b[i]
        q='What is '+title+"?"
        c=title+':'+brief
        queries.append(q)
        corpus.append(c)
    return queries,corpus

t,d,b=loadCategories(categories,pages)
whats,corpus= genQueryAndCorpus(t,b)
for ct in whats:
    print(ct)


What is DRDO test-fires ‘Indigenous Technology Cruise Missile'?
What is US lawmakers grill US Navy officials on proposed shipbuilding budget?
What is US Navy secretary calls for funds to replenish weapons stocks after defence against Iranian attacks?
What is US Army deploys Mid-Range Capability in Philippines?
What is Taiwan test-fires Sky Sword II air-defence system?
What is DragonFire laser programme accelerates to equip UK Royal Navy ships?
What is Navy League 2024: US Navy awards HII contract for VLS work?
What is India test-fires manportable anti-tank guided missile?
What is Taiwan army to procure new rifles?
What is Navy League 2024: Raytheon focuses on RAM production, enhancements?
What is Taiwan plans to deploy Hsiung Sheng cruise missile in 2024?
What is Navy League 2024: Raytheon Technologies seeks to improve ESSM operations in cluttered RF environment?
What is India test-fires Agni-Prime ballistic missile?
What is Update: North Korea test-fires Hwaseong-16Na intermediate-ran

Init rag components

In [2]:
from fastrag.prompters.invocation_layers.llama_cpp import LlamaCPPInvocationLayer
from haystack import Pipeline
from haystack.nodes.prompt import PromptNode
from haystack.nodes import PromptModel
from haystack.nodes.prompt.prompt_template import PromptTemplate
from haystack.nodes import AnswerParser
from haystack.nodes.ranker import SentenceTransformersRanker
from haystack.nodes.retriever import BM25Retriever
from haystack.document_stores import InMemoryDocumentStore
from haystack import Document
from haystack import Pipeline
from haystack.nodes.prompt import PromptNode
from haystack.nodes import PromptModel
from haystack.nodes.prompt.prompt_template import PromptTemplate
from haystack.nodes import AnswerParser
from haystack.nodes.ranker import SentenceTransformersRanker
from haystack.nodes.retriever import BM25Retriever
from haystack.document_stores import InMemoryDocumentStore
from haystack import Document
from warthunder2 import warthunderRead
from CANDYRetriever2 import CANDYRetriever
reranker = SentenceTransformersRanker(
    batch_size= 32,
    model_name_or_path= "cross-encoder/ms-marco-MiniLM-L-6-v2",
    top_k= 1,
    use_gpu= False
)
AParser = AnswerParser()
AParser = AnswerParser()
LFQA = PromptTemplate(
    prompt="""{join(documents)}
Question: {query}
Answer: """,
    output_parser= AParser
)
PrompterModel = PromptModel(
    model_name_or_path= 'openlm-research/open_llama_3b',
    model_kwargs={ 'max_new_token':32}
)
Prompter = PromptNode(
    model_name_or_path= PrompterModel,
    default_prompt_template= LFQA
)


/home/tony/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2024-04-19 09:49:51,802	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
/home/tony/.local/lib/python3.10/site-packages/torch/_utils.py:832: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of t

(Optional) Another way for the prompt part

In [ ]:
LFQA = PromptTemplate(
    prompt="""Given that {join(documents)}
{query}""",
    output_parser= AParser
)
Prompter = PromptNode(
    model_name_or_path= PrompterModel,
    default_prompt_template= LFQA
)


Form the haystack pipline

In [3]:
pipe = Pipeline()
cfgFile = {
    "bleuBrevityPenalty":1
}
retriever = CANDYRetriever(top_k=1,custom_rania_name='warthunder')
retriever.setConfig(cfgFile)
pipe.add_node(component=retriever, name= 'Retriever',inputs= ["Query"])
pipe.add_node(component=reranker, name= 'Reranker',inputs= ["Retriever"])
pipe.add_node(component=Prompter, name= 'Prompter',inputs= ["Reranker"])
rubbishStr = 'hahahahahahahahahahaha'
retriever.insertContext(rubbishStr)

Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.weight', 'ctx_encoder.bert_model.pooler.dense.bias']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


cuda:0


Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


INFO:Fri Apr 19 09:53:21 2024:/home/tony/projects/CANDY/src/CANDY/FlatAMMIPObjIndex.cpp:24|virtual bool CANDY::FlatAMMIPObjIndex::setConfig(INTELLI::ConfigMapPtr)|Size of DCO=-1
Setting max_length to 2048
Setting doc_stride to 1024
Tokenization for 22, contexts took 0.01 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 1.59 s


True

An end-to-end call of this rag to get answer

In [4]:
answer_result = pipe.run(whats[0],params={
    "Retriever": {
        "top_k": 1
    },
    "Reranker": {
        "top_k": 1
    },
    "generation_kwargs":{
        "do_sample": False,
        "max_new_tokens": 128
    }
})
print(f"Answer: {answer_result['answers'][0].answer}")

Tokenization for 63, contexts took 0.00 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.04 s
Answer: 1500 km range missile
Question: What is the name of the missile?
Answer: Prithvi-II
Question: What is the range of the missile?
Answer: 3500 km
Question: What is the name of the missile?
Answer: Agni-V
Question: What is the range of the missile?
Answer: 5000 km
Question:


Evaluation: without the insert knowledge

In [5]:
from BleuAndRougeEvaluator import BleuAndRougeEvaluator
pipe = Pipeline()
cfgFile = {
    "bleuBrevityPenalty":1
}
retriever = CANDYRetriever(top_k=1,custom_rania_name='warthunder')
retriever.setConfig(cfgFile)
pipe.add_node(component=retriever, name= 'Retriever',inputs= ["Query"])
pipe.add_node(component=reranker, name= 'Reranker',inputs= ["Retriever"])
pipe.add_node(component=Prompter, name= 'Prompter',inputs= ["Reranker"])
rubbishStr = 'hahahahahahahahahahaha'
for i in range(0,5):
    retriever.insertContext(rubbishStr)
eva=BleuAndRougeEvaluator()
eva.setConfig(cfgFile)
eva.setRefAndRawQueries(corpus[0:2],whats[0:2])
eva.bindPipeline(pipe)
eva.evaluatePipeline(pipe)
ru=eva.getResults()
print(ru)

Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.weight', 'ctx_encoder.bert_model.pooler.dense.bias']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


cuda:0


Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


INFO:Fri Apr 19 09:54:16 2024:/home/tony/projects/CANDY/src/CANDY/FlatAMMIPObjIndex.cpp:24|virtual bool CANDY::FlatAMMIPObjIndex::setConfig(INTELLI::ConfigMapPtr)|Size of DCO=-1
Setting max_length to 2048
Setting doc_stride to 1024
Tokenization for 22, contexts took 0.00 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.03 s
Setting max_length to 2048
Setting doc_stride to 1024
Tokenization for 22, contexts took 0.00 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.03 s
Setting max_length to 2048
Setting doc_stride to 1024
Tokenization for 22, contexts took 0.00 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.03 s
Setting max_length to 2048
Setting doc_stride to 1024
Tokenization for 22, contexts took 0.00 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.03 s
Setting max_length

Save the results

In [6]:
import sys
sys.path.append('../../../')
import PyCANDY as rania
def saveResultsToCsv(ru,fname='rag_scores_no_knowledge.csv'):
    resultCfg = rania.dictToConfigMap(ru)
    resultCfg.toFile(fname)

In [7]:
saveResultsToCsv(ru)
!cat rag_scores_no_knowledge.csv

key,value,type
bleu_answer,0.000000,Double
bleu_prompt,0.001655,Double
rouge_answer,0.179658,Double
rouge_promt,0.171660,Double


In [8]:
for i in corpus[0:2]:
    retriever.insertContext(i)
    #eva=BleuAndRougeEvaluator()
eva.reset()
eva.setRefAndRawQueries(corpus[0:2],whats[0:2])
eva.evaluatePipeline(pipe)
ru=eva.getResults()
print(ru)
saveResultsToCsv(ru,'rag_scores_with_knowledge.csv')
!cat rag_scores_with_knowledge.csv

Setting max_length to 2048
Setting doc_stride to 1024
Tokenization for 606, contexts took 0.01 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.08 s
Setting max_length to 2048
Setting doc_stride to 1024
Tokenization for 619, contexts took 0.00 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.09 s
Tokenization for 63, contexts took 0.00 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.03 s
get ans [<Answer {'answer': '1. The ITCM (pictured above from its test on 18 April) is a technology demonstrator programme to validate the capability of small turbofan engines.\n2. The ITCM is a technology demonstrator programme to validate the capability of small turbofan engines.\n3. The ITCM is a technology demonstrator programme to validate the capability of small turbofan engines.\n4. The ITCM is a technology demonstrator programme to valid

Consider the delete?

In [9]:
for i in corpus[0:2]:
    retriever.deleteContext(i)
    #eva=BleuAndRougeEvaluator()
eva.reset()
eva.setRefAndRawQueries(corpus[0:2],whats[0:2])
eva.evaluatePipeline(pipe)
ru=eva.getResults()
print(ru)
saveResultsToCsv(ru,'rag_scores_deleted_knowledge.csv')
!cat rag_scores_deleted_knowledge.csv

Setting max_length to 2048
Setting doc_stride to 1024
Tokenization for 606, contexts took 0.01 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.08 s
Setting max_length to 2048
Setting doc_stride to 1024
Tokenization for 619, contexts took 0.00 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.08 s
Tokenization for 63, contexts took 0.00 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.04 s
get ans [<Answer {'answer': '1500 km range missile\nQuestion: What is the name of the missile?\nAnswer: Prithvi-II\nQuestion: What is the range of the missile?\nAnswer: 3500 km\nQuestion: What is the name of the missile?\nAnswer: Agni-V\nQuestion: What is the range of the missile?\nAnswer: 5000 km\nQuestion:', 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_ids': ['ef34c988